# Word Embeddings e Sentiment Analysis (PyTorch puro)

Este notebook demonstra como representar palavras por vetores densos (*word embeddings*) e usá-los em uma tarefa de classificação de sentimentos usando apenas PyTorch.

Etapas:
1. Preparar um pequeno corpus rotulado (positivo/negativo)
2. Tokenizar e mapear palavras para índices
3. Criar embeddings aprendíveis
4. Treinar um classificador simples
5. Avaliar e testar previsões

In [152]:
# pip install torch torchvision torchaudio

In [153]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

## 1. Conjunto de dados simples

In [154]:
corpus = [
    ("a película foi ótima e emocionante", 1),
    ("o filme foi ótimo e emocionante", 1),
    ("adorei o enredo e os personagens", 1),
    ("a atuação foi excelente", 1),
    ("o filme é terrível e cansativo", 0),
    ("péssimo roteiro e trilha sonora ruim", 0),
    ("não gostei do final", 0),
]

## 2. Tokenização e vocabulário

In [155]:
def tokenize(text):
    return text.lower().split()

vocab = {"<pad>": 0, "<unk>": 1}
for sentence, _ in corpus:
    for tok in tokenize(sentence):
        if tok not in vocab:
            vocab[tok] = len(vocab)

vocab_size = len(vocab)
print("Vocabulário:", vocab)

def encode(sentence, max_len=8):
    tokens = tokenize(sentence)
    ids = [vocab.get(t, 1) for t in tokens]
    if len(ids) < max_len:
        ids += [0] * (max_len - len(ids))
    else:
        ids = ids[:max_len]
    return ids

max_len = 8
encoded = [(encode(s, max_len), y) for s, y in corpus]
print(encoded[:2])

Vocabulário: {'<pad>': 0, '<unk>': 1, 'a': 2, 'película': 3, 'foi': 4, 'ótima': 5, 'e': 6, 'emocionante': 7, 'o': 8, 'filme': 9, 'ótimo': 10, 'adorei': 11, 'enredo': 12, 'os': 13, 'personagens': 14, 'atuação': 15, 'excelente': 16, 'é': 17, 'terrível': 18, 'cansativo': 19, 'péssimo': 20, 'roteiro': 21, 'trilha': 22, 'sonora': 23, 'ruim': 24, 'não': 25, 'gostei': 26, 'do': 27, 'final': 28}
[([2, 3, 4, 5, 6, 7, 0, 0], 1), ([8, 9, 4, 10, 6, 7, 0, 0], 1)]


## 3. Dataset e DataLoader

In [156]:
class SentimentDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        x, y = self.data[idx]
        return torch.tensor(x), torch.tensor(y, dtype=torch.float32)

dataset = SentimentDataset(encoded)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

## 4. Modelo com embeddings aprendíveis

In [157]:
class EmbeddingClassifier(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.fc1 = nn.Linear(emb_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)
        self.act = nn.ReLU()
        self.out = nn.Sigmoid()

    def forward(self, x):
        emb = self.emb(x)
        mean_emb = emb.mean(dim=1)
        h = self.act(self.fc1(mean_emb))
        y = self.out(self.fc2(h))
        #return y.squeeze()
        return y.view(-1)


model = EmbeddingClassifier(vocab_size, emb_dim=16, hidden_dim=8)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

## 5. Treinamento

In [158]:
for epoch in range(25):
    total_loss = 0
    for xb, yb in loader:
        optimizer.zero_grad()
        preds = model(xb) # forward pass
        print(preds.shape, yb.shape)
        loss = criterion(preds, yb) # loss
        loss.backward() # backpropagation
        optimizer.step() # parameter update
        total_loss += loss.item()
    if (epoch+1) % 5 == 0:
        print(f"Época {epoch+1:02d} | Loss: {total_loss/len(loader):.4f}")

torch.Size([2]) torch.Size([2])
torch.Size([2]) torch.Size([2])
torch.Size([2]) torch.Size([2])
torch.Size([1]) torch.Size([1])
torch.Size([2]) torch.Size([2])
torch.Size([2]) torch.Size([2])
torch.Size([2]) torch.Size([2])
torch.Size([1]) torch.Size([1])
torch.Size([2]) torch.Size([2])
torch.Size([2]) torch.Size([2])
torch.Size([2]) torch.Size([2])
torch.Size([1]) torch.Size([1])
torch.Size([2]) torch.Size([2])
torch.Size([2]) torch.Size([2])
torch.Size([2]) torch.Size([2])
torch.Size([1]) torch.Size([1])
torch.Size([2]) torch.Size([2])
torch.Size([2]) torch.Size([2])
torch.Size([2]) torch.Size([2])
torch.Size([1]) torch.Size([1])
Época 05 | Loss: 0.5927
torch.Size([2]) torch.Size([2])
torch.Size([2]) torch.Size([2])
torch.Size([2]) torch.Size([2])
torch.Size([1]) torch.Size([1])
torch.Size([2]) torch.Size([2])
torch.Size([2]) torch.Size([2])
torch.Size([2]) torch.Size([2])
torch.Size([1]) torch.Size([1])
torch.Size([2]) torch.Size([2])
torch.Size([2]) torch.Size([2])
torch.Size([2]) 

## 6. Teste com novas frases

In [159]:
def predict(sentence):
    x = torch.tensor(encode(sentence, max_len)).unsqueeze(0)
    with torch.no_grad():
        p = model(x).item()
    label = "positivo" if p >= 0.5 else "negativo"
    return f"{sentence} → {label} ({p:.2f})"

for s in [
    "amei o filme e a trilha sonora",
    "o roteiro é fraco e entediante",
    "personagens excelentes e cativantes",
    "não recomendo o filme",
]:
    print(predict(s))

amei o filme e a trilha sonora → negativo (0.05)
o roteiro é fraco e entediante → negativo (0.25)
personagens excelentes e cativantes → positivo (0.97)
não recomendo o filme → negativo (0.22)


## 7. Visualizando embeddings

In [160]:
emb_weights = model.emb.weight.detach()
print("Dimensão:", emb_weights.shape)

def closest(word, topn=5):
    if word not in vocab: 
        return []
    idx = vocab[word]
    wv = emb_weights[idx]
    # similaridade de cosseno em PyTorch
    sims = torch.nn.functional.cosine_similarity(emb_weights, wv.unsqueeze(0))
    best = torch.argsort(sims, descending=True)[1:topn+1]
    inv_vocab = {i: w for w, i in vocab.items()}
    return [(inv_vocab[int(i)], float(sims[i])) for i in best]

print("Mais próximos de 'filme':", closest("filme"))


Dimensão: torch.Size([29, 16])
Mais próximos de 'filme': [('<pad>', 0.3396492600440979), ('trilha', 0.3035092055797577), ('final', 0.2873351275920868), ('do', 0.21413035690784454), ('terrível', 0.1970217525959015)]
